In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch.nn.functional as F
import math
import bz2
from collections import Counter
import re
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm
import gc

In [ ]:
print('Чтение файлов')

In [ ]:
file = pd.DataFrame()
for i in range(10): # 9
    tf = pd.read_csv('/kaggle/input/mts-ml-cup-rnn-features/{}.parquet_rnn.csv'.format(i))
    file = file.append(tf)
    
file.drop(['Unnamed: 0'], axis=1, inplace=True)
file.columns = ['user_id', 'seq']

In [ ]:
file

In [ ]:
print('Создание словаря больших векторов сайтов')

In [ ]:
y = np.load('/kaggle/input/mts-ml-cup-sites-clusterization/y.npy')
sites_raw = pd.read_excel('/kaggle/input/mts-ml-cup-sites-raw/sites.xlsx')

In [ ]:
y_mean = y.mean(axis=0)

dict_vectors = {}

for index, row in tqdm(sites_raw.iterrows()):
    try:
        dict_vectors[row['url_host']] = y[index]
    except:
        continue

In [ ]:
del y, tf
gc.collect()

In [ ]:
print('Векторизация последовательностей посещений на больших векторах')

In [ ]:
# Первый вариант. Учитываем каждое посещение сайта

vector_sentences = []

sentences = list(file.seq.values)

for i, sentence in enumerate(tqdm(sentences)):
    vec = np.zeros((768,))
    counter = 1
    try:
        for el in sentence.split(','):
            if el in dict_vectors:
                vec += dict_vectors[el]
                counter += 1
        if vec.sum() != 0:
            vector_sentences.append(vec / counter)
        else:
            vector_sentences.append(y_mean)
    except:
        vector_sentences.append(y_mean)
        
print(np.array(vector_sentences).shape)
np.save('emb1.npy', np.array(vector_sentences))
np.savez_compressed('emb1_compressed', a=np.array(vector_sentences))

In [ ]:
del vector_sentences
gc.collect()

In [ ]:
# Второй вариант. Учитываем уникальные сайты

vector_sentences = []

sentences = list(file.seq.values)

for i, sentence in enumerate(tqdm(sentences)):
    vec = np.zeros((768,))
    counter = 1
    try:
        for el in set(sentence.split(',')):
            if el in dict_vectors:
                vec += dict_vectors[el]
                counter += 1
        if vec.sum() != 0:
            vector_sentences.append(vec / counter)
        else:
            vector_sentences.append(y_mean)
    except:
        vector_sentences.append(y_mean)
        
print(np.array(vector_sentences).shape)
np.save('emb2.npy', np.array(vector_sentences))
np.savez_compressed('emb2_compressed', a=np.array(vector_sentences))

In [ ]:
del vector_sentences
gc.collect()

In [ ]:
print('Создание словаря сжатых векторов сайтов')

y = np.load('/kaggle/input/mts-ml-cup-sites-clusterization/y_umap.npy')

y_mean = y.mean(axis=0)

dict_vectors = {}

for index, row in tqdm(sites_raw.iterrows()):
    try:
        dict_vectors[row['url_host']] = y[index]
    except:
        continue

del y
gc.collect()

In [ ]:
print('Векторизация последовательностей посещений на сжатых векторах')

In [ ]:
# Первый вариант. Учитываем каждое посещение сайта

vector_sentences = []

sentences = list(file.seq.values)

for i, sentence in enumerate(tqdm(sentences)):
    vec = np.zeros((24,))
    counter = 1
    try:
        for el in sentence.split(','):
            if el in dict_vectors:
                vec += dict_vectors[el]
                counter += 1
        if vec.sum() != 0:
            vector_sentences.append(vec / counter)
        else:
            vector_sentences.append(y_mean)
    except:
        vector_sentences.append(y_mean)
        
print(np.array(vector_sentences).shape)
np.save('emb3.npy', np.array(vector_sentences))
np.savez_compressed('emb3_compressed', a=np.array(vector_sentences))

In [ ]:
del vector_sentences
gc.collect()

In [ ]:
# Второй вариант. Учитываем уникальные сайты

vector_sentences = []

sentences = list(file.seq.values)

for i, sentence in enumerate(tqdm(sentences)):
    vec = np.zeros((24,))
    counter = 1
    try:
        for el in set(sentence.split(',')):
            if el in dict_vectors:
                vec += dict_vectors[el]
                counter += 1
        if vec.sum() != 0:
            vector_sentences.append(vec / counter)
        else:
            vector_sentences.append(y_mean)
    except:
        vector_sentences.append(y_mean)
        
print(np.array(vector_sentences).shape)
np.save('emb4.npy', np.array(vector_sentences))
np.savez_compressed('emb4_compressed', a=np.array(vector_sentences))

In [ ]:
del vector_sentences
gc.collect()

In [ ]:
file[['user_id']].to_csv('user_id.csv', index=False)